# NIDM: A Summary in many parts

## Part 3: Querying NIDM with SPARQL

satra@mit.edu, nolan.nichols@gmail.com

[Latest version](https://github.com/INCF/ni-/notebooks/master/NIDMQuery.ipynb)

## Outline

- Related notebooks
- Querying object models with [SPARQL](http://www.w3.org/TR/rdf-sparql-query/) using a [triplestore](http://en.wikipedia.org/wiki/Triplestore)

## Related notebooks



### Querying object models with SPARQL using a triplestore

In this section we will show example SPARQL queries that access a triplestore.

    - What is a triplestore?
    - How are NI-DM objects uploaded to a triplestore?
    - How do we query a triplestore for data stored using NI-DM?

### What is a triplestore?

    - A triplestore is a database for storing RDF triples (i.e., subject, predicate, object statements). 
    - The triplestore provides a way to create, read, update, and delete persisted RDF statements using the SPARQL 1.1 protocol. 
    - Here we are using the triplestore functionality of the Virtuoso database (http://virtuoso.openlinksw.com/) 
    - The specific triplestore can easily be swapped for another system such as Jena or Seseme.

### How are NIDM objects uploaded to a triplestore?

    - The examples above demonstrates how to create a collection of NI-DM FreeSurfer objects in RDF.
    - RDF files can be bulk uploaded through the Virtuoso web interface or using a SPARQL Insert Statement
    - This is an example of adding a single RDF triple

In [ ]:
import requests
from requests.auth import HTTPDigestAuth

from IPython.core.display import HTML

"""
    This is just an example
"""


# connection params for secure endpoint
endpoint = 'http://computor.mit.edu:8890/sparql'
username = 'username'
password = 'password'

# session defaults
session = requests.Session()
session.auth = HTTPDigestAuth(username, password)

session.headers = {'Accept':'text/html'} # HTML from SELECT queries

# INSERT - CREATE new triples
query =  """PREFIX nidm: <http://nidm.nidash.org#>
            PREFIX prov: <http://www.w3.org/ns/prov#>
            PREFIX nlx: <http://uri.neuinfo.org/nif/nifstd/>
            
            INSERT DATA
            INTO GRAPH <http://nidm.nidash.org>
                {nidm:UUID12345 a prov:Entity;
                    nlx:nlx_150774 "2.0"}"""

data = {'query': query}
result = session.post(endpoint, data=data)
HTML(result.content)

### How do we query a triplestore for data stored using NI-DM?

    - this example shows how to query for FreeSurfer details given a subject id

In [ ]:
    def getSubjectDetails(self, subject_id):
        session = requests.Session()

        qstring = '''
        PREFIX fs: <http://surfer.nmr.mgh.harvard.edu/fswiki/#>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        PREFIX nidm: <http://nidm.nidash.org/#>
        PREFIX prov: <http://www.w3.org/ns/prov#>

        select distinct ?structureName ?structureGV where {
             ?subjectCollection fs:subject_id "%s"^^<http://www.w3.org/2001/XMLSchema#string> .
             ?subjectCollection prov:hadMember ?otherMembers .
             ?collectionFromProv prov:wasDerivedFrom ?otherMembers .
             ?collectionFromProv prov:hadMember ?membersOfProvCollection .
             ?membersOfProvCollection a fs:GrayVol . # filter by those that have a type of fs:GrayVol(ume)
             ?membersOfProvCollection fs:structure ?structureName . 
             ?membersOfProvCollection fs:value ?structureGV .
        } ''' % (subject_id)